# 4.0.1 Reformatting the Breakfast Dataset

Changing the format from avis to something more amenable to models.

This might be needed later

In [1]:
!conda install h5py

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.7.12
  latest version: 4.8.2

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /media/data/conda/abdullah/envs/bk

  added / updated specs:
    - h5py


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    h5py-2.8.0                 |   py36h989c5e5_3         911 KB  defaults
    ------------------------------------------------------------
                                           Total:         911 KB

The following NEW packages will be INSTALLED:

  h5py               pkgs/main/linux-64::h5py-2.8.0-py36h989c5e5_3



h5py-2.8.0   

## Jupyter Extensions

Load [watermark](https://github.com/rasbt/watermark) to see the state of the machine and environment that's running the notebook. To make sense of the options, take a look at the [usage](https://github.com/rasbt/watermark#usage) section of the readme.

In [3]:
# Load `watermark` extension
%load_ext watermark
# Display the status of the machine and packages. Add more as necessary.
%watermark -v -n -m -g -b -t -p torch,torchvision,cv2,h5py,matplotlib,seaborn,jupyterlab,lab

Mon Feb 17 2020 17:06:44 

CPython 3.6.10
IPython 7.12.0

torch 1.3.1
torchvision 0.1.8
cv2 3.4.2
h5py 2.8.0
matplotlib 3.1.3
seaborn 0.10.0
jupyterlab 1.2.6
lab 0+untagged.16.gfa80751.dirty

compiler   : GCC 7.3.0
system     : Linux
release    : 4.15.0-72-generic
machine    : x86_64
processor  : x86_64
CPU cores  : 16
interpreter: 64bit
Git hash   : fa80751f80d2efa12599315e9e67f52d745180b0
Git branch : master


Load [autoreload](https://ipython.org/ipython-doc/3/config/extensions/autoreload.html) which will always reload modules marked with `%aimport`.

This behavior can be inverted by running `autoreload 2` which will set everything to be auto-reloaded *except* for modules marked with `%aimport`.

In [4]:
# Load `autoreload` extension
%load_ext autoreload
# Set autoreload behavior
%autoreload 1

Load `matplotlib` in one of the more `jupyter`-friendly [rich-output modes](https://ipython.readthedocs.io/en/stable/interactive/plotting.html). Some options (that may or may not have worked) are `inline`, `notebook`, and `gtk`.

In [5]:
# Set the matplotlib mode
%matplotlib inline

## Set the GPU

Make sure we aren't greedy.

In [6]:
!nvidia-smi

Mon Feb 17 17:07:33 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 390.30                 Driver Version: 390.30                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  TITAN Xp            Off  | 00000000:04:00.0 Off |                  N/A |
| 23%   27C    P8    16W / 250W |      0MiB / 12196MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  TITAN Xp            Off  | 00000000:05:00.0 Off |                  N/A |
| 23%   31C    P8    16W / 250W |      0MiB / 12196MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  T

In [7]:
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_VISIBLE_DEVICES=0


## Imports

In [43]:
from pathlib import Path

import cv2
import numpy as np
from tqdm import tqdm

Local imports that may or may not be autoreloaded. This section contains things that will likely have to be re-imported multiple times, and have additions or subtractions made throughout the project.

In [11]:
# Constants to be used throughout the package
%aimport lab.index
from lab.index import DIR_DATA_INT
# Import the data subdirectories
%aimport lab.breakfast.index
from lab.breakfast.index import DIR_BREAKFAST_VIDEOS, DIR_COARSE_SEG, DIR_FINE_SEG

## Saving All AVI Frames

### Testing out Saving Functionality

In [14]:
# Create the landing dir
flattened_images = DIR_DATA_INT / DIR_BREAKFAST_VIDEOS.stem / 'flattened_images'

if not flattened_images.exists():
    flattened_images.mkdir(parents=True)

In [15]:
path = next(iter(DIR_BREAKFAST_VIDEOS.rglob('*.avi')))

In [16]:
path

PosixPath('/media/data_cifs/apra/work/labwork/data/external/breakfast/BreakfastII_15fps_qvga_sync/P39/cam01/P39_salat.avi')

In [17]:
cap = cv2.VideoCapture(str(path))
ret, frame = cap.read()

In [18]:
frame.shape

(240, 320, 3)

In [19]:
type(frame)

numpy.ndarray

In [30]:
path.parts.index(DIR_BREAKFAST_VIDEOS.stem)

9

In [40]:
img_name = flattened_images / '_'.join(path.parts[path.parts.index(DIR_BREAKFAST_VIDEOS.stem)+1:])[:-4]
img_name

PosixPath('/media/data_cifs/apra/work/labwork/data/interim/BreakfastII_15fps_qvga_sync/flattened_images/P39_cam01_P39_salat')

In [38]:
cv2.imwrite(str(img_name) + '_test_01.png', frame)

True

### Creating the Images

In [45]:
completed = []

for path in tqdm(DIR_BREAKFAST_VIDEOS.rglob('*.avi')):
    if path in completed:
        print(f'Path {path} found. Skipping.')
        continue
    
    name_root = flattened_images / '_'.join(path.parts[
        path.parts.index(DIR_BREAKFAST_VIDEOS.stem)+1:])[:-4]
    cap = cv2.VideoCapture(str(path))
    
    ret = True
    i = 0
    while ret:
        ret, frame = cap.read()
        cv2.imwrite(str(name_root) + f'_{i}.png', frame)
        i += 1
        
    completed.append(path)
    print(f'Completed {path}!')


0it [00:00, ?it/s]
1it [00:44, 44.41s/it]

Completed /media/data_cifs/apra/work/labwork/data/external/breakfast/BreakfastII_15fps_qvga_sync/P39/cam01/P39_salat.avi!



2it [01:03, 36.76s/it]

Completed /media/data_cifs/apra/work/labwork/data/external/breakfast/BreakfastII_15fps_qvga_sync/P39/cam01/P39_tea.avi!



3it [03:09, 63.67s/it]

...


12it [10:27, 53.70s/it]

Completed /media/data_cifs/apra/work/labwork/data/external/breakfast/BreakfastII_15fps_qvga_sync/P39/webcam01/P39_friedegg.avi!



13it [10:51, 45.02s/it]

Completed /media/data_cifs/apra/work/labwork/data/external/breakfast/BreakfastII_15fps_qvga_sync/P39/webcam01/P39_sandwich.avi!


KeyboardInterrupt: 

This was taking too long and had already generated 40GB of data.

## Subsampling

In [46]:
len(list(DIR_BREAKFAST_VIDEOS.rglob('*.avi')))

1989

In [48]:
len(list(DIR_BREAKFAST_VIDEOS.rglob('*webcam01*/*.avi')))

365

Decided to go with some of the precomputed features on the main website.